In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import joblib

# ====== Definir Modelo ======
class LinkMLP(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

# ====== Carregar modelo ======
model = LinkMLP()
model.load_state_dict(torch.load("../models/mlp.pth", weights_only=True))  # só os pesos
model.eval()

scaler = joblib.load("../models/scaler.pkl")

# ====== Dados de teste ======
# [rssi, etx, delay, busy_fraction]
test_data = [
    [-55, 2, 17, 0.3],
    [-85, 3.8, 30, 0.1],
    [-35, 1.2, 5, 0.05],
    [-88, 9.5, 95, 0.9],
    [-40, 8.2, 85, 0.8],
    [-87, 1.5, 8, 0.15],
    [-62, 2.8, 32, 0.28],
    [-58, 3.2, 28, 0.32],
    [-52, 2.1, 20, 0.25],
    [-48, 2.5, 25, 0.22],
    [-50, 1.8, 22, 0.28],
    [-75, 5.2, 65, 0.6],
    [-78, 4.8, 70, 0.65],
    [-72, 5.5, 60, 0.58],
    [-45, 4.5, 15, 0.7],
    [-70, 1.9, 45, 0.2],
    [-60, 2.9, 30, 0.3],
    [-65, 2.2, 18, 0.25],
    [-50, 3.5, 35, 0.35],    
    [-32, 1.1, 3, 0.02],
    [-90, 9.8, 98, 0.95],
    [-38, 1.3, 7, 0.08],
    [-86, 8.5, 88, 0.85],
    [-42, 1.6, 12, 0.12],    
    [-33, 7.2, 75, 0.8],
    [-82, 1.2, 8, 0.1],
    [-75, 8.1, 5, 0.85],
    [-68, 6.5, 85, 0.05],    
    [-89, 2.1, 18, 0.15],
    [-45, 9.2, 22, 0.2],
    [-52, 2.3, 95, 0.25],
    [-48, 2.0, 20, 0.88],    
    [-59, 3.0, 29, 0.29],
    [-61, 2.9, 31, 0.31],
    [-60.5, 3.05, 30.5, 0.305],
    [-59.5, 2.95, 29.5, 0.295],    
    [-67, 4.2, 45, 0.5],
    [-64, 3.8, 42, 0.48],
    [-69, 4.5, 48, 0.52],
    [-66, 4.0, 40, 0.45],    
    [-54, 1.9, 16, 0.28],
    [-56, 2.2, 19, 0.26],
    [-51, 1.7, 21, 0.24],
    [-49, 2.4, 18, 0.27],
    [-53, 2.0, 23, 0.29],    
    [-76, 5.8, 67, 0.63],
    [-74, 5.1, 62, 0.59],
    [-77, 6.2, 72, 0.68],
    [-73, 5.5, 58, 0.56],
    [-79, 5.9, 75, 0.71],    
    [-55, 1.8, 35, 0.6],
    [-45, 4.2, 25, 0.7],
    [-65, 2.5, 18, 0.55],
    [-58, 3.8, 28, 0.8],    
    [-41, 5.0, 50, 0.5],
    [-80, 2.0, 20, 0.2],
    [-60, 6.0, 60, 0.6],
    [-50, 2.5, 25, 0.25],    
    [-57, 2.8, 28, 0.28],
    [-63, 3.2, 32, 0.32],
    [-58, 2.9, 31, 0.29],
    [-62, 3.1, 29, 0.31],    
    [-30, 1.0, 1, 0.01],
    [-90, 10.0, 100, 1.0],
    [-31, 1.5, 10, 0.05],
    [-89, 9.0, 90, 0.9],    
    [-44, 7.5, 12, 0.15],
    [-83, 1.8, 78, 0.75],
    [-38, 6.8, 8, 0.82],
    [-71, 1.4, 55, 0.18]
]

X_test = torch.tensor(scaler.transform(test_data), dtype=torch.float)

# ====== Inferência ======
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(test_data):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"Aresta {row} -> Classe {cls} ({CLASS_NAMES[cls]}) (confiança {conf:.2f})")

Aresta [-55, 2, 17, 0.3] -> Classe 1 (Bom) (confiança 0.99)
Aresta [-85, 3.8, 30, 0.1] -> Classe 1 (Bom) (confiança 0.66)
Aresta [-35, 1.2, 5, 0.05] -> Classe 1 (Bom) (confiança 1.00)
Aresta [-88, 9.5, 95, 0.9] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-40, 8.2, 85, 0.8] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-87, 1.5, 8, 0.15] -> Classe 1 (Bom) (confiança 0.95)
Aresta [-62, 2.8, 32, 0.28] -> Classe 1 (Bom) (confiança 0.94)
Aresta [-58, 3.2, 28, 0.32] -> Classe 1 (Bom) (confiança 0.94)
Aresta [-52, 2.1, 20, 0.25] -> Classe 1 (Bom) (confiança 1.00)
Aresta [-48, 2.5, 25, 0.22] -> Classe 1 (Bom) (confiança 1.00)
Aresta [-50, 1.8, 22, 0.28] -> Classe 1 (Bom) (confiança 1.00)
Aresta [-75, 5.2, 65, 0.6] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-78, 4.8, 70, 0.65] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-72, 5.5, 60, 0.58] -> Classe 0 (Ruim) (confiança 1.00)
Aresta [-45, 4.5, 15, 0.7] -> Classe 1 (Bom) (confiança 0.72)
Aresta [-70, 1.9, 45, 0.2] -> Classe 1 (Bom) (confiança 0.92